In [1]:
import yfinance as yf
import pandas as pd
import time
from datetime import datetime

# ✅ Full US Stock Universe (Sector-Wise 50 Stocks)
us_stock_universe = {
    "Technology": ["AAPL", "MSFT", "GOOGL", "NVDA", "META"],
    "Banking & Financials": ["JPM", "BAC", "WFC", "C", "GS"],
    "Consumer Goods": ["PG", "KO", "PEP", "MO", "MNST"],
    "Pharma & Healthcare": ["JNJ", "PFE", "MRNA", "LLY", "UNH"],
    "Energy": ["XOM", "CVX", "COP", "PSX", "SLB"],
    "Automobile": ["TSLA", "F", "GM", "RIVN", "LCID"],
    "Retail & E-commerce": ["AMZN", "WMT", "COST", "TGT", "HD"],
    "Aerospace & Defense": ["BA", "LMT", "NOC", "GD", "RTX"],
    "Telecom & Media": ["VZ", "T", "DIS", "NFLX", "CMCSA"],
    "Semiconductors": ["AMD", "TSM", "INTC", "AVGO", "QCOM"]
}

# ✅ Data Storage List
stock_data = []
fetch_date = datetime.now().strftime('%Y-%m-%d')  # ✅ Get current date

# ✅ Fetch Data for Each Stock
for sector, stocks in us_stock_universe.items():
    for stock in stocks:
        try:
            ticker = yf.Ticker(stock)

            # ✅ Fetch Last Closing Price
            hist = ticker.history(period="1d")
            last_price = hist["Close"].iloc[-1] if not hist.empty else "N/A"

            # ✅ Fetch Current P/E Ratio
            current_pe = ticker.info.get("trailingPE", "N/A")

            # ✅ Fetch Historical Avg P/E Ratio (Past 3 Years)
            eps_values = ticker.financials.loc["Diluted EPS"].dropna() if "Diluted EPS" in ticker.financials.index else pd.Series()
            avg_pe = (last_price / eps_values).mean() if not eps_values.empty else "N/A"

            # ✅ Fetch Direct Net Worth (Shareholder's Equity)
            net_worth = ticker.balance_sheet.loc["Total Stockholder Equity"].iloc[0] if "Total Stockholder Equity" in ticker.balance_sheet.index else "N/A"

            # ✅ If Direct Net Worth is N/A, Approximate It
            if net_worth == "N/A":
                market_cap = ticker.info.get("marketCap", "N/A")
                price_to_book = ticker.info.get("priceToBook", "N/A")

                if market_cap != "N/A" and price_to_book != "N/A" and price_to_book > 0:
                    net_worth = market_cap / price_to_book  # **Estimate Net Worth**
                else:
                    net_worth = "N/A"

            # ✅ Convert Net Worth to Billion $
            net_worth_billion = round(net_worth / 1e9, 2) if net_worth != "N/A" else "N/A"

            # ✅ Store Data
            stock_data.append({
                "Date Fetched": fetch_date,
                "Sector": sector,
                "Stock": stock,
                "Last Price ($)": round(last_price, 2) if last_price != "N/A" else "N/A",
                "Current P/E Ratio": round(current_pe, 2) if current_pe != "N/A" else "N/A",
                "Historical Avg P/E Ratio": round(avg_pe, 2) if avg_pe != "N/A" else "N/A",
                "Net Worth ($B)": net_worth_billion
            })

            print(f"✅ {stock} - Date: {fetch_date}, Price: {last_price}, P/E: {current_pe}, Avg P/E: {avg_pe}, Net Worth: {net_worth_billion}")

        except Exception as e:
            print(f"❌ Error fetching {stock}: {e}")

        time.sleep(1)  # ✅ API Rate Limit Avoid Karne ke liye

# ✅ Convert to Pandas DataFrame
df = pd.DataFrame(stock_data)

# ✅ Print as Table
print(df)

# ✅ Save to CSV File (Optional)
df.to_csv("stock_prices.csv", index=False)

✅ AAPL - Date: 2025-04-06, Price: 188.3800048828125, P/E: 29.949127, Avg P/E: 31.531283602010745, Net Worth: 66.67


✅ MSFT - Date: 2025-04-06, Price: 359.8399963378906, P/E: 28.995972, Avg P/E: 37.414551934790865, Net Worth: 302.65


✅ GOOGL - Date: 2025-04-06, Price: 145.60000610351562, P/E: 18.109453, Avg P/E: 33.16782169838194, Net Worth: 326.67


✅ NVDA - Date: 2025-04-06, Price: 94.30999755859375, P/E: 32.078228, Avg P/E: 227.76401334792615, Net Worth: 79.08


✅ META - Date: 2025-04-06, Price: 504.7300109863281, P/E: 21.14495, Avg P/E: 37.62720852033674, Net Worth: 182.61


...

✅ TSM - Date: 2025-04-06, Price: 146.8000030517578, P/E: 21.588236, Avg P/E: 0.8954754284582792, Net Worth: 857.71


✅ INTC - Date: 2025-04-06, Price: 19.850000381469727, P/E: N/A, Avg P/E: 16.989885208076643, Net Worth: 99.97


✅ AVGO - Date: 2025-04-06, Price: 146.2899932861328, P/E: 67.41474, Avg P/E: 79.08569081236254, Net Worth: 278.45


✅ QCOM - Date: 2025-04-06, Price: 127.45999908447266, P/E: 13.749729, Avg P/E: 15.367262927586157, Net Worth: 26.89


   Date Fetched                Sector  Stock  Last Price ($)  \
0    2025-04-06            Technology   AAPL          188.38   
1    2025-04-06            Technology   MSFT          359.84   
2    2025-04-06            Technology  GOOGL          145.60   
3    2025-04-06            Technology   NVDA           94.31   
4    2025-04-06            Technology   META          504.73   
5    2025-04-06  Banking & Financials    JPM          210.28   
6    2025-04-06  Banking & Financials    BAC           34.39   
7    2025-04-06  Banking & Financials    WFC           60.98   
8    2025-04-06  Banking & Financials      C           58.13   
9    2025-04-06  Banking & Financials     GS          470.81   
10   2025-04-06        Consumer Goods     PG          163.75   
11   2025-04-06        Consumer Goods     KO           69.93   
12   2025-04-06        Consumer Goods    PEP          146.61   
13   2025-04-06        Consumer Goods     MO           56.07   
14   2025-04-06        Consumer Goods   

In [2]:
# Define stock universe
us_stock_universe = {
    "Technology": ["AAPL", "MSFT", "GOOGL", "NVDA", "META"],
    "Banking & Financials": ["JPM", "BAC", "WFC", "C", "GS"],
    "Consumer Goods": ["PG", "KO", "PEP", "MO", "MNST"],
    "Pharma & Healthcare": ["JNJ", "PFE", "MRNA", "LLY", "UNH"],
    "Energy": ["XOM", "CVX", "COP", "PSX", "SLB"],
    "Automobile": ["TSLA", "F", "GM", "RIVN", "LCID"],
    "Retail & E-commerce": ["AMZN", "WMT", "COST", "TGT", "HD"],
    "Aerospace & Defense": ["BA", "LMT", "NOC", "GD", "RTX"],
    "Telecom & Media": ["VZ", "T", "DIS", "NFLX", "CMCSA"],
    "Semiconductors": ["AMD", "TSM", "INTC", "AVGO", "QCOM"]
}

# Function to fetch sector-wise stock data
def get_sector_stock_data():
    sector_data = {}

    for sector, tickers in us_stock_universe.items():
        sector_prices = {"Last Price": [], "Open Price": [], "Highest": [], "Lowest": []}

        for ticker in tickers:
            stock = yf.Ticker(ticker)
            hist = stock.history(period="1mo")  # Fetch latest monthly data

            if not hist.empty:
                sector_prices["Last Price"].append(hist["Close"].iloc[-1])
                sector_prices["Open Price"].append(hist["Open"].iloc[-1])
                sector_prices["Highest"].append(hist["High"].iloc[-1])
                sector_prices["Lowest"].append(hist["Low"].iloc[-1])

        # Calculate average prices for the sector
        if sector_prices["Last Price"]:
            sector_data[sector] = {
                "Avg Last Price": sum(sector_prices["Last Price"]) / len(sector_prices["Last Price"]),
                "Avg Open Price": sum(sector_prices["Open Price"]) / len(sector_prices["Open Price"]),
                "Avg Highest": sum(sector_prices["Highest"]) / len(sector_prices["Highest"]),
                "Avg Lowest": sum(sector_prices["Lowest"]) / len(sector_prices["Lowest"]),
            }
        else:
            sector_data[sector] = ["Data not available"]

    return pd.DataFrame(sector_data).T

# Fetch sector data
sector_stock_data = get_sector_stock_data()

# Rename the index to "Sector"
sector_stock_data.index.name = "Sector"

# Display and save the data
print(sector_stock_data)
sector_stock_data.to_csv("sector_data.csv", index=True)

print("✅ Sector-wise stock data saved to sector_stock_data.csv")


                      Avg Last Price  Avg Open Price  Avg Highest  Avg Lowest
Sector                                                                       
Technology                258.572003      262.312000   268.734001  255.702005
Banking & Financials      166.917999      172.120000   173.895999  163.723998
Consumer Goods             98.688000      102.868002   104.072002   98.502001
Pharma & Healthcare       292.916003      302.209001   305.940004  291.836206
Energy                     93.499998       98.755999    99.418002   92.368001
Automobile                 61.335999       64.534001    65.836000   60.070000
Retail & E-commerce       324.057996      330.067996   338.474004  320.490005
Aerospace & Defense       284.343994      299.551999   301.964006  283.382002
Telecom & Media           208.487997      218.038000   220.750001  207.923999
Semiconductors            105.232000      109.039999   110.650001  102.811999
✅ Sector-wise stock data saved to sector_stock_data.csv


In [3]:
import yfinance as yf
import pandas as pd

# Define stock universe
us_stock_universe = {
    "Technology": ["AAPL", "MSFT", "GOOGL", "NVDA", "META"],
    "Banking & Financials": ["JPM", "BAC", "WFC", "C", "GS"],
    "Consumer Goods": ["PG", "KO", "PEP", "MO", "MNST"],
    "Pharma & Healthcare": ["JNJ", "PFE", "MRNA", "LLY", "UNH"],
    "Energy": ["XOM", "CVX", "COP", "PSX", "SLB"],
    "Automobile": ["TSLA", "F", "GM", "RIVN", "LCID"],
    "Retail & E-commerce": ["AMZN", "WMT", "COST", "TGT", "HD"],
    "Aerospace & Defense": ["BA", "LMT", "NOC", "GD", "RTX"],
    "Telecom & Media": ["VZ", "T", "DIS", "NFLX", "CMCSA"],
    "Semiconductors": ["AMD", "TSM", "INTC", "AVGO", "QCOM"]
}

# Function to fetch sector-wise averages
def get_sector_averages():
    sector_data = {}
    
    for sector, tickers in us_stock_universe.items():
        pe_ratios = []
        hist_avg_pe_ratios = []
        net_worths = []
        
        for ticker in tickers:
            stock = yf.Ticker(ticker)
            try:
                pe_ratio = stock.info.get("trailingPE")
                hist_avg_pe = stock.info.get("forwardPE")
                net_worth = stock.info.get("marketCap") / 1e9 if stock.info.get("marketCap") else None
                
                if pe_ratio is not None:
                    pe_ratios.append(pe_ratio)
                if hist_avg_pe is not None:
                    hist_avg_pe_ratios.append(hist_avg_pe)
                if net_worth is not None:
                    net_worths.append(net_worth)
            except:
                continue
        
        sector_data[sector] = {
            "Avg P/E Ratio": sum(pe_ratios) / len(pe_ratios) if pe_ratios else None,
            "Avg Historical P/E": sum(hist_avg_pe_ratios) / len(hist_avg_pe_ratios) if hist_avg_pe_ratios else None,
            "Total Net Worth ($B)": sum(net_worths) if net_worths else None,
        }
    
    return pd.DataFrame(sector_data).T

# Fetch sector averages
sector_avg_data = get_sector_averages()

# Rename index to "Sector"
sector_avg_data.index.name = "Sector"

# Display and save data
print(sector_avg_data)
sector_avg_data.to_csv("sector_avg_data.csv", index=True)

print("✅ Sector-wise average P/E, historical P/E, and net worth saved to sector_avg_data.csv")


                      Avg P/E Ratio  Avg Historical P/E  Total Net Worth ($B)
Sector                                                                       
Technology                26.055546           21.165844          10871.513022
Banking & Financials      10.819876           10.495691           1304.268800
Consumer Goods            24.487460           20.721408           1036.066566
Pharma & Healthcare       34.883266           13.909571           1652.440779
Energy                    14.019627           11.374202            901.670425
Automobile                43.621642           15.374008            871.772978
Retail & E-commerce       30.704494           27.350390           3281.150624
Aerospace & Defense       21.982996           71.612785            498.328412
Telecom & Media           21.310101           16.164255           1015.685325
Semiconductors            47.128176           17.915944           1816.090485
✅ Sector-wise average P/E, historical P/E, and net worth saved t